In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the California housing dataset
california = fetch_california_housing()

# Split the data into features and target
X = california.data
y = california.target

print(X.shape)
print(y.shape)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

(20640, 8)
(20640,)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

Train_loader = DataLoader(list(zip(X_train, y_train)), shuffle=True, batch_size=128)
Test_loader = DataLoader(list(zip(X_test, y_test)), shuffle=False, batch_size=16)

Training_loss=[]
Validation_loss=[]
Training_accuracy=[]
Validation_accuracy=[]

# Define the MultiLayer Perceptron model
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 6)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(6, 4)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(4, 2)
        self.relu = nn.ReLU()
        self.fc4 = nn.Linear(2,1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        return x

# Function to calculate accuracy
def accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    total = labels.size(0)
    correct = (predicted == labels).sum().item()
    return 100 * correct / total

# Function to calculate precision, recall, and f1-score
def precision_recall(model, criterion, optimizer, Test_loader):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in Test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted',labels=np.unique(y_pred))
    return precision, recall, f1


# Function to train the model
def train(model, criterion, optimizer, Train_loader):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in Train_loader:
        optimizer.zero_grad()
        labels =  labels.type(torch.LongTensor)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    epoch_loss = running_loss / len(Train_loader)
    epoch_accuracy = 100 * correct / total
    Training_loss.append(epoch_loss)
    Training_accuracy.append(epoch_accuracy)
    return epoch_loss, epoch_accuracy

# Function to validate the model
def validate(model, criterion, Test_loader):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in Test_loader:
            labels =  labels.type(torch.LongTensor)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    epoch_loss = running_loss / len(Test_loader)
    epoch_accuracy = 100 * correct / total
    Validation_loss.append(epoch_loss)
    Validation_accuracy.append(epoch_accuracy)
    return epoch_loss, epoch_accuracy

# Define hyperparameters
input_size = 8                                                                  # Enter the input size (number of features)                                                                 # Enter the number of hidden units                                                         # Number of classes in the dataset
lr = 0.001                                                                      # Learning rate
num_epochs = 80

# Initialize the model, loss function, and optimizer
model = MLP(input_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# Training loop
for epoch in range(num_epochs):
    train_loss, train_accuracy = train(model, criterion, optimizer, Train_loader)
    val_loss, val_accuracy = validate(model, criterion, Test_loader)
    precision, recall, _ = precision_recall(model, criterion, optimizer, Test_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}] - "
          f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, "
          f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%, "
          f"Precision: {precision:.4f}, Recall: {recall:.4f}")

# Calculate and print confusion matrix
y_true = []
y_pred = []
with torch.no_grad():
    for inputs, labels in Test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: ignored

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing

# Load the California housing dataset
housing = fetch_california_housing()
data = housing.data
target = housing.target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

# Standardize the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Define the regression model
class RegressionModel(nn.Module):
    def __init__(self, input_size):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 6)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(6, 4)
        self.fc3 = nn.Linear(4, 2)
        self.fc4 = nn.Linear(2, 1)  # Output layer with one neuron (regression)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        return x

# Create the model
input_size = X_train_scaled.shape[1]
model = RegressionModel(input_size)

# Loss function (Mean Squared Error) and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 1500
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    # Calculate MAE and RMSE for evaluation
    with torch.no_grad():
        predicted = model(X_test_tensor)
        mae = mean_absolute_error(y_test, predicted.numpy())
        rmse = mean_squared_error(y_test, predicted.numpy(), squared=False)

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}, MAE: {mae:.4f}, RMSE: {rmse:.4f}")

# Evaluate the model on test data
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)
    mae = mean_absolute_error(y_test, test_outputs.numpy())
    rmse = mean_squared_error(y_test, test_outputs.numpy(), squared=False)

print(f'Test Loss: {test_loss.item():.4f}, Test MAE: {mae:.4f}, Test RMSE: {rmse:.4f}')


Epoch [1/1500], Loss: 3.9462, MAE: 1.5964, RMSE: 1.9595
Epoch [2/1500], Loss: 3.9256, MAE: 1.5903, RMSE: 1.9543
Epoch [3/1500], Loss: 3.9051, MAE: 1.5843, RMSE: 1.9492
Epoch [4/1500], Loss: 3.8847, MAE: 1.5783, RMSE: 1.9440
Epoch [5/1500], Loss: 3.8643, MAE: 1.5723, RMSE: 1.9389
Epoch [6/1500], Loss: 3.8441, MAE: 1.5663, RMSE: 1.9338
Epoch [7/1500], Loss: 3.8239, MAE: 1.5603, RMSE: 1.9287
Epoch [8/1500], Loss: 3.8039, MAE: 1.5543, RMSE: 1.9236
Epoch [9/1500], Loss: 3.7839, MAE: 1.5484, RMSE: 1.9185
Epoch [10/1500], Loss: 3.7641, MAE: 1.5424, RMSE: 1.9134
Epoch [11/1500], Loss: 3.7443, MAE: 1.5364, RMSE: 1.9083
Epoch [12/1500], Loss: 3.7246, MAE: 1.5305, RMSE: 1.9033
Epoch [13/1500], Loss: 3.7051, MAE: 1.5246, RMSE: 1.8983
Epoch [14/1500], Loss: 3.6856, MAE: 1.5187, RMSE: 1.8932
Epoch [15/1500], Loss: 3.6662, MAE: 1.5128, RMSE: 1.8882
Epoch [16/1500], Loss: 3.6469, MAE: 1.5069, RMSE: 1.8832
Epoch [17/1500], Loss: 3.6278, MAE: 1.5011, RMSE: 1.8782
Epoch [18/1500], Loss: 3.6087, MAE: 1.49